In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade --force-reinstall --no-cache-dir torch==2.1.0 triton --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"
!pip install unsloth
!pip install datasets
!pip install nltk


Looking in indexes: https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0+cu121, 2.2.1+cu121, 2.2.2+cu121, 2.3.0+cu121, 2.3.1+cu121, 2.4.0+cu121, 2.4.1+cu121, 2.5.0+cu121, 2.5.1+cu121)
ERROR: No matching distribution found for torch==2.1.0


  Cloning https://github.com/unslothai/unsloth.git to c:\users\hp\appdata\local\temp\pip-install-xr6pgl8q\unsloth_5d11647cda664448addaeacb2a694811


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


  Using cached importlib_metadata-8.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   ---------------------------------------- 992.0/992.0 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/167.7 MB ? eta -:--:--
   ---------------------------------------- 1.3/167.7 MB 6.1 MB/s eta 0:00:28
    --------------------------------------- 2.6/167.7 MB 6.6 MB/s eta 0:00:26
   - -------------------------------------- 4.2/167.7 MB 6.6 MB/s eta 0:00:25
   - -------------------------------------- 5.8/167.7 MB 6.9 MB/s eta 0:00:24
   - -------------------------------------- 7.3/167.7 MB 7.1 MB/s eta 0:00:23
   -- ------------------------------------- 8.9/167.7 MB 7.1 MB/s eta 0:00:23
   -- ------------------------------------- 10.5/167.7 MB 7.1 MB/s eta 0:00:23
   -- ------------------------------------- 11.8/167.7 MB 7.2 MB/s eta 0:00:22
   --- --------

In [3]:
!pip install evaluate


In [4]:
!pip install rouge_score


In [5]:
import evaluate
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")


C:\Users\HP\anaconda3\envs\experimentation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'float8_e4m3b11fnuz' from 'tensorflow.python.framework.dtypes' (C:\Users\HP\anaconda3\envs\experimentation\Lib\site-packages\tensorflow\python\framework\dtypes.py)

In [ ]:
!pip install tqdm


In [ ]:
from unsloth import FastLanguageModel
import pandas as pd
from transformers import TrainingArguments, pipeline
import torch
import numpy as np
import evaluate  # Updated import for metrics
from tqdm import tqdm  # Import tqdm for progress bars

# Load the pre-trained Mistral model
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=5048,  # Maximum sequence length
    dtype=None,  # Use float16 for a balance between performance and precision
    load_in_4bit=True,   # Load in full precision for better accuracy
)
print("Mistral model loaded successfully.")

# Display some information about the model
print("\nModel Information:")
print(f"Model Name: {model_name}")
print(f"Vocabulary Size: {len(tokenizer)}")
print(f"Maximum Sequence Length: {model.config.max_position_embeddings}")

# Calculate and display the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params:,}")
print("Model information displayed successfully.")

# Prepare the Alpaca prompt for training
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
print("Alpaca prompt for training prepared.")

# Enable native 2x faster inference for model
FastLanguageModel.for_inference(model)
print("Enabled faster inference for the model.")

# Tokenizing an example question
inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is (are) Glaucoma ?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
print("Example question tokenized successfully.")

# Generate a sample text
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1000)
print("Sample text generated successfully.")

# Load and process the CSV file
data = pd.read_csv('/kaggle/input/layoutlm/medquad.csv')
data = data.iloc[:100, :2]  # Keep only the first 100 rows for demonstration
data.columns = ['question', 'answer']
print("CSV data loaded and processed.")

# Convert the DataFrame to a Hugging Face Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(data)
print("DataFrame converted to Hugging Face Dataset.")

# Preprocessing function to format data according to Alpaca prompt
def preprocess_function(examples):
    alpaca_prompts = [f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n\n### Input:\n{input_text}\n\n### Response:\n" for question, input_text in zip(examples['question'], examples['answer'])]
    examples['text'] = [prompt + response for prompt, response in zip(alpaca_prompts, examples['answer'])]
    return examples

# Apply the preprocessing function to the dataset
dataset = dataset.map(preprocess_function, batched=True, remove_columns=['question', 'answer'])
print("Preprocessing function applied successfully.")

# Split the dataset into training and validation sets (80% train, 20% validation)
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(eval_dataset)}")
print("Dataset split into training and validation sets.")

# Training Arguments
training_args = TrainingArguments(
   per_device_train_batch_size = 4,
   gradient_accumulation_steps = 4,
   warmup_steps = 5,
   num_train_epochs=1,
   max_steps=-1,
   learning_rate = 2e-4,
   fp16 = not torch.cuda.is_bf16_supported(),
   bf16 = torch.cuda.is_bf16_supported(),
   logging_steps = 1,
   optim = "adamw_8bit",
   weight_decay = 0.01,
   lr_scheduler_type = "linear",
   seed = 3407,
   output_dir = "outputs",
   report_to = "none",
)
print("Training arguments set successfully.")

# Patching model with LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,  # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
print("Model patched with LoRA successfully.")

from trl import SFTTrainer

# Trainer setup
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences.
    args = training_args,
)
print("Trainer setup completed.")

# Show memory stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
print("Memory stats before training displayed.")

# Train the model with a progress bar
with tqdm(total=training_args.num_train_epochs, desc="Training Progress") as pbar:
    for epoch in range(training_args.num_train_epochs):
        trainer.train()
        pbar.update(1)  # Update the progress bar after each epoch
print("Model training completed.")



In [ ]:
def evaluate_model(model, tokenizer, eval_dataset):
    # Initialize the evaluation metrics using 'evaluate' library
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")
    print("Evaluation metrics loaded successfully.")

    # Initialize the model for text generation
    text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    # List to store predicted and reference outputs
    predictions = []
    references = []

    # Preprocess and tokenize the inputs
    for example in eval_dataset:
        input_text = example['text']

        # Tokenize the input text
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

        # Generate predictions from the model
        generated_text = text_generator(input_text, max_new_tokens=100, num_return_sequences=1)[0]['generated_text']

        # Store the predicted and reference texts
        predictions.append(generated_text)
        references.append(input_text)

    # Compute BLEU score
    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
    print(f"BLEU Score: {bleu_score}")

    # Compute ROUGE score
   ## rouge_score = rouge_metric.compute(predictions=predictions, references=references)
    ##print(f"ROUGE Score: {rouge_score}")

    # Perplexity calculation (can be done using the loss on the eval dataset)
   ## eval_loss = trainer.evaluate(eval_dataset=eval_dataset)['eval_loss']
    ###perplexity = np.exp(eval_loss)
    ###print(f"Perplexity: {perplexity}")

# Perform evaluation on the validation set
evaluate_model(model, tokenizer, eval_dataset)

### Model saving and loading

In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu


In [ ]:
!pip install chromadb


### RAG INCLUDD IN TH CODE BELOW

In [ ]:
!pip install --upgrade --force-reinstall --no-cache-dir torch==2.1.0 triton --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import chromadb
from chromadb.config import Settings
from transformers import RagTokenizer, RagRetriever, pipeline
from unsloth import FastLanguageModel  # Your pre-trained PEFT model
import torch
import evaluate  # Ensure you have the 'evaluate' library for BLEU
import faiss
import numpy as np

# Assuming your pre-trained PEFT model is already loaded like this
# and has been trained using FastLanguageModel and LoRA.
# You can skip re-loading it here.

# Use the already trained model
print("Using pre-trained model.")

# Set the device for model and input tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available)
model = model.to(device)
print("Model moved to device:", device)

# Set pad_token_id if it's not already set (necessary for RAG model)
if not hasattr(tokenizer, 'pad_token_id') or tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id  # Or you can use 0
print("Pad token ID set successfully.")

# Initialize Chroma client for document retrieval
chroma_client = chromadb.Client()
print("Chroma client initialized successfully.")

# Initialize a collection for storing documents (Chroma will store documents and embeddings)
try:
    chroma_client.delete_collection(name="rag_retriever")  # Delete the collection if it exists
    print("Existing collection 'rag_retriever' deleted.")
except Exception as e:
    print(f"Error deleting collection: {e}")

collection = chroma_client.create_collection(name="rag_retriever")
print("Chroma collection created.")

# Example of adding documents (replace with your actual document data)
documents = ["What is the capital of France?", "How far is the moon from the Earth?"]

# Use the tokenizer to encode documents and get embeddings
inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate embeddings using the model output logits
with torch.no_grad():
    outputs = model.model(**inputs)  # Forward pass
    # Extract logits or hidden states
    embeddings = outputs.logits.mean(dim=1).cpu().numpy()  # Use mean pooling over the sequence length

# Store the documents and their embeddings in Chroma
for i, doc in enumerate(documents):
    collection.add(
        documents=[doc],
        metadatas=[{"source": f"doc{i}"}],
        ids=[f"doc{i}"],
        embeddings=embeddings[i]  # Use actual embeddings
    )

# Set up the retriever using Chroma
class ChromaRetriever:
    def __init__(self, chroma_client, collection_name):
        self.collection = chroma_client.get_collection(collection_name)

    def retrieve(self, query, top_k=5):
        query_input = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Move the query input tensors to the same device as the model
        query_input = {key: value.to(device) for key, value in query_input.items()}

        # Generate query embedding using the same process as for documents
        with torch.no_grad():
            query_outputs = model.model(**query_input)  # Forward pass
            query_embedding = query_outputs.logits.mean(dim=1).cpu().numpy()  # Mean pooling
        
        # Query the Chroma collection using the generated query embedding
        results = self.collection.query(
            query_embeddings=query_embedding,
            n_results=top_k
        )
        return results['documents']

# Initialize ChromaRetriever
chroma_retriever = ChromaRetriever(chroma_client, "rag_retriever")
print("Chroma retriever initialized successfully.")

# Now, we integrate the trained PEFT model with the RAG architecture
rag_model = model  # Use the already trained model as the RAG model
print("Retriever set for the RAG model using Chroma.")

# Define a custom function to evaluate the RAG model
def evaluate_rag_model(model, tokenizer, retriever, eval_dataset):
    # Initialize the evaluation metrics using 'evaluate' library
    bleu_metric = evaluate.load("bleu")
    print("Evaluation metrics loaded for RAG model.")

    # Initialize the model for text generation using the RAG model
    text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    # List to store predicted and reference outputs
    predictions = []
    references = []

    for example in eval_dataset:
        input_text = example['text']

        # Retrieve relevant documents using Chroma retriever
        retrieved_docs = retriever.retrieve(input_text, top_k=3)  # Retrieve top 3 documents

        # Generate predictions from the RAG model
        generated_text = text_generator(input_text, max_new_tokens=100, num_return_sequences=1)[0]['generated_text']

        # Store the predicted and reference texts
        predictions.append(generated_text)
        references.append(example['text'])

    # Compute BLEU score for the RAG model
    bleu_score = bleu_metric.compute(predictions=predictions, references=references)
    print(f"RAG Model BLEU Score: {bleu_score}")

# Example eval_dataset (replace with your actual dataset)
eval_dataset = train_test_split['test']
# Perform evaluation on the RAG model using the validation set
evaluate_rag_model(rag_model, tokenizer, chroma_retriever, eval_dataset)
print("Evaluation on RAG model completed.")
